<a href="https://colab.research.google.com/github/ramincre/Dual-Path-Transformer-Network-PyTorch/blob/main/BSS_DPTNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
!cp "/content/drive/MyDrive/Dataset/MiniLibriMix.rar" "/content"

In [ ]:
!unrar x /content/MiniLibriMix.rar

In [5]:
import os

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
!git clone https://github.com/ramincre/Dual-Path-Transformer-Network-PyTorch.git

Cloning into 'Dual-Path-Transformer-Network-PyTorch'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 298 (delta 66), reused 129 (delta 63), pack-reused 162
Receiving objects: 100% (298/298), 2.61 MiB | 16.03 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [8]:
!mkdir /content/output

In [9]:
!python /content/Dual-Path-Transformer-Network-PyTorch/preprocess.py --in-dir /content/MiniLibriMix-Clean --out-dir /content/output --sample-rate 8000

Namespace(in_dir='/content/MiniLibriMix-Clean', out_dir='/content/output', sample_rate=8000)


In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/train.py \
--train_dir '/content/output/tr'\
--valid_dir '/content/output/cv'\
--epochs 20 \
--batch_size 2 \
--num_workers 0  \
--N 48\
--momentum 0.01\
--l2 0.001

Namespace(train_dir='/content/output/tr', valid_dir='/content/output/cv', sample_rate=8000, segment=4, cv_maxlen=8, N=48, C=2, L=4, H=4, K=250, B=6, use_cuda=1, epochs=20, half_lr=0, early_stop=0, max_norm=5, shuffle=0, batch_size=2, num_workers=0, optimizer='adam', lr=0.125, momentum=0.01, l2=0.001, save_folder='exp/temp', checkpoint=0, continue_from='', model_path='final.pth.tar', print_freq=1000, visdom=0, visdom_epoch=0, visdom_id='TasNet training')
Drop 38 utts(0.00 h) which is short than 32000 samples
# of parameters: 1577089
Training...
Epoch 1 | Iter 1 | Average Loss 19.190 | Current Loss 19.189829 | 7935.3 ms/batch
-------------------------------------------------------------------------------------
Train Summary | End of Epoch 1 | Time 86.46s | Train Loss 17.504
-------------------------------------------------------------------------------------
Cross validation...
Epoch 1 | Iter 1 | Average Loss 15.952 | Current Loss 15.951954 | 269.0 ms/batch
------------------------------

In [ ]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [ ]:
!pip install mir_eval

In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/evaluate.py \
--data_dir '/content/output/tt' \
--model_path '/content/exp/temp/temp_best.pth.tar'\
--batch_size 2\
--epochs 20 \
--batch_size 2 \
--num_workers 0  \
--N 48\
--momentum 0.01\


In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/separate.py\
 --mix_json '/content/output/tt/mix.json' \
 --model_path '/content/exp/temp/temp_best.pth.tar'\
 --N 48 \
--batch_size 2